## Text Mining 

Objectif : Fonction qui renvoie pour chaque numéro de cluster un mot qui décrit le mieux le cluster

Input : dataframe 
Output : cluster avec le title 

In [ ]:
import pandas as pd
df = pd.read_parquet("flickr_data_clustered.parquet")

On commence par supprimer les stopwords

In [ ]:
#fonction
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re


french_stopwords = {
    "le","la","les","de","des","du","un","une","et","à","au","aux",
    "en","pour","sur","avec","sans","ce","cette","ces","dans",
    "est","sont","été","être","avoir","Lyon","Demeure","Chaos" ,"France"
}

stopwords = ENGLISH_STOP_WORDS.union(french_stopwords)


def clean_stopwords(text) : 

    if not isinstance(text, str):
        return ""
    
    words = text.split()
    words = [w for w in words if w not in stopwords and len(w) > 2]
    return words
    

#apply the function to the 'title' column
df['cleaned_title_stopwords'] = df['title'].apply(clean_stopwords)

df['cleaned_tags_stopwords'] = df['tags'].apply(clean_stopwords)
   

In [ ]:
df

Ensuite on on cherche le mot qui correspond à chaque cluster
df : le dataframe 
cluster_kmeans : le nom de la colonne qui contient le num de cluster auquel appartient la ligne
texte_cols : les colonnes où y a les textes qu'on va utiliser pour identifier les mots les plus fréquent 
top_k : nb de mots à garder = 1 puisque on cherche un seul mots par cluster

In [ ]:
from collections import Counter

def cluster_titles(
    df,
    cluster_kmeans="cluster_kmeans",
    text_cols=("cleaned_title_stopwords", "cleaned_tags_stopwords"),
    top_k=1
):
    """
    Retourne un titre (mot-clé) par cluster
    """
    cluster_labels = {}

    for cluster_id in sorted(df[cluster_kmeans].unique()):
        if cluster_id == -1:
            continue  # on ignore le bruit

        # sous-dataframe du cluster (seulement les ligne de ce cluster)
        dff = df[df[cluster_kmeans] == cluster_id]

        # concaténation des textes de toutes les lignes
        all_words = []
        #on parcours les deux colonnes de texte
        for col in text_cols:
            texts = dff[col].dropna()
            for t in texts:
                all_words.extend(t)
        # si la liste est vide donc pas de mots
        if not all_words:
            cluster_labels[cluster_kmeans] = "unknown"
            continue

        # comptage des mots
        counts = Counter(all_words)

        # mots les plus fréquents
        top_words = [w for w, _ in counts.most_common(top_k)]

        cluster_labels[cluster_id] = top_words

    return cluster_labels

df['cluster_name'] = df['cluster_kmeans'].map(
    cluster_titles(df, top_k=1)
)

In [ ]:
df